In [ ]:
from fastapi import FastAPI, Request, HTTPException
import asyncio
import json
import logging
from datetime import datetime

app = FastAPI()

# Configuração de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Simulação de um banco de dados em memória para armazenar callbacks recebidos
CALLBACK_STORAGE = []

@app.post("/callbacks")
async def receive_callback(request: Request):
    """
    Endpoint que recebe callbacks da BMP.
    """
    try:
        # Lê os dados recebidos
        callback_data = await request.json()

        # Validação básica
        if "Header" not in callback_data or "Body" not in callback_data:
            raise HTTPException(status_code=400, detail="Formato inválido")

        # Armazena a transação para processamento assíncrono
        CALLBACK_STORAGE.append(callback_data)

        # Log para monitoramento
        logging.info(f"Callback recebido: {json.dumps(callback_data, indent=2, ensure_ascii=False)}")

        # Retorna sucesso
        return {"status": "200 OK", "message": "Callback recebido com sucesso"}

    except Exception as e:
        logging.error(f"Erro ao processar callback: {str(e)}")
        raise HTTPException(status_code=400, detail="Erro no processamento do callback")

# Simulação de processamento assíncrono dos callbacks recebidos
async def process_callbacks():
    while True:
        if CALLBACK_STORAGE:
            callback = CALLBACK_STORAGE.pop(0)
            logging.info(f"Processando callback: {callback}")
            # Simule um delay de processamento
            await asyncio.sleep(2)
            logging.info(f"Callback {callback['Body']['IDTransacao']} processado com sucesso!")
        await asyncio.sleep(1)  # Intervalo entre verificações

# Inicia a tarefa assíncrona ao subir o servidor
@app.on_event("startup")
async def startup_event():
    asyncio.create_task(process_callbacks())